In [ ]:
import os
%pwd

In [2]:
os.chdir("../")

In [ ]:
%pwd

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

@dataclass(frozen=True)
class DataEmbeddingConfig:
    root_dir: Path
    data_path: Path
    model_name: str
    text_column: str # 要進行 embedding 的欄位名稱

In [ ]:
from dialogue_rag_chatbot.constants import *
from dialogue_rag_chatbot.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

    # def get_data_loading_config(self) -> DataLoadingConfig:
    #     config = self.config.data_loading
        
    #     create_directories([config.root_dir])

    #     data_validation_config = DataLoadingConfig(
    #         root_dir=config.root_dir,
    #         ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
    #     )

    #     return data_validation_config
    
    def get_data_embedding_config(self) -> DataEmbeddingConfig:
        config = self.config.data_embedding

        create_directories([config.root_dir])
        data_embedding_config = DataEmbeddingConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_name = config.model_name,
            text_column= config.text_column
        )

        return data_embedding_config

In [ ]:
import os
from dialogue_rag_chatbot.logging import logger
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from typing import List, Dict, Any, Tuple

c:\Users\Kun\miniconda3\envs\legalrag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-09-26 13:05:14,057: INFO: utils: NumExpr defaulting to 16 threads.]


In [9]:
# class DataLoading:
#     def __init__(self, config: DataLoadingConfig):
#         self.config = config # path
    
#     def _convert_dataset(self, dataset_json, file_type: str):
#         """
#         Convert HuggingFace dataset to list of dicts with content and metadata
        
#         Args:
#             dataset_json: HuggingFace dataset object (DatasetDict)
#             file_type: str, e.g. "train", "test", "validation"

#         Returns:
#             List[dict]
#         """
#         documents = []
#         split_name = list(dataset_json.keys())[0]
#         dataset = dataset_json[split_name]

#         # for row in dataset:
#         #     documents.append({
#         #         "content": row["dialogue"],
#         #         "metadata": {
#         #             "source": row["id"],
#         #             "file_type": file_type,
#         #             "summary": row["summary"]
#         #         }
#         #     })

#         return documents


#     def load_docs(self):
#         """
#         Load documents from file paths
            
#         Returns:
#             List of document dictionaries with content and metadata
#         """
#         documents = []
#         # Obtain all file under directory of artifacts/data_ingestion 
#         all_files = os.listdir(os.path.join("artifacts", "data_ingestion"))
#         print(f'all files: {all_files}')

#         for required_file in self.config.ALL_REQUIRED_FILES:
#             try:
#                 if required_file in all_files:
#                     path = os.path.join("artifacts", "data_ingestion", required_file)
#                     dataset_json = load_dataset("json", data_files=path)
#                     file_type = os.path.splitext(required_file)[0]

#                     converted = self._convert_dataset(dataset_json, file_type)
#                     documents.extend(converted)
#             except Exception as e:
#                 logger.error(f"Error loading {path}: {str(e)}")
                
#         logger.info(f"Total documents loaded: {len(documents)}")
#         return documents
###########################################################################################################

class EmbeddingModel:
    """Granite embedding model using SentenceTransformers"""
    
    def __init__(self, config: DataEmbeddingConfig):
        self.model_name = config.model_name
        try:
            self.model = SentenceTransformer(self.model_name)
            logger.info(f"Granite embedding model '{self.model_name}' loaded successfully")
        except Exception as e:
            logger.error(f"Failed to load model {self.model_name}: {str(e)}")
            raise
    
    def encode(self, texts):
        """Encode texts into embeddings"""
        try:
            # Granite embedding models return 768-dimensional vectors
            embeddings = self.model.encode(texts, convert_to_numpy=True)
            logger.info(f"Encoded {len(texts)} texts into embeddings of shape {embeddings.shape}")
            return embeddings
        except Exception as e:
            logger.error(f"Error encoding texts: {str(e)}")
            raise


class DataEmbeddingTransformation:
    """
    Transforms text data into embeddings using a specified model.
    """
    def __init__(self, config: DataEmbeddingConfig):
        self.config = config
        self.embedding_model = EmbeddingModel(config=config)

    def _generate_embeddings(self, batch: Dict[str, List]):
        """
        Takes a batch of examples and generates embeddings for the specified text column.
        This method is designed to be used with dataset.map().
        """
        texts = batch[self.config.text_column]
        embeddings = self.embedding_model.encode(texts)
        # 返回一個字典，key 是新的欄位名，value 是 embedding 列表
        return {"embedding": embeddings.tolist()}

    def convert(self, save_path: str = "samsum_dataset"):
        """
        Loads the dataset, generates embeddings for the text column,
        and saves the processed dataset to disk.
        """
        logger.info(f"Loading dataset from '{self.config.data_path}'...")
        # 載入資料集，這裡只取訓練集的前100筆作為範例
        dataset = load_dataset(self.config.data_path, split="train") 
        # dataset = load_dataset(self.config.data_path) 
        logger.info(f"Generating embeddings for column '{self.config.text_column}'...")
        # 使用 .map() 將 embedding 函數應用於整個資料集
        # batched=True 和適當的 batch_size 可以大幅提升處理速度
        dataset_with_embeddings = dataset.map(
            self._generate_embeddings, 
            batched=True, 
            batch_size=32  # 可根據您的硬體調整
        )
        
        # 儲存處理完畢的資料集
        output_path = os.path.join(self.config.root_dir, save_path)
        logger.info(f"Saving dataset with embeddings to '{output_path}'...")
        dataset_with_embeddings.save_to_disk(output_path)
        
        logger.info("Data embedding transformation complete.")
        return dataset_with_embeddings



In [ ]:
# try:
#     config = ConfigurationManager()
#     data_loading_config = config.get_data_loading_config()
#     data_loading = DataLoading(config=data_loading_config)
#     data_contents = data_loading.load_docs()
    
# except Exception as e:
#     raise e

[2025-09-25 22:06:55,455: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-25 22:06:55,470: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-25 22:06:55,473: INFO: common: created directory at: artifacts]
[2025-09-25 22:06:55,475: INFO: common: created directory at: artifacts/data_loading]
all files: ['data.7z', 'licence.txt', 'README.txt', 'test.json', 'train.json', 'val.json']
[2025-09-25 22:06:56,768: INFO: 3729515940: Total documents loaded: 819]


In [10]:
try:
    # 2. 執行資料轉換
    config = ConfigurationManager()
    data_embedding_config = config.get_data_embedding_config()
    transformer = DataEmbeddingTransformation(config=data_embedding_config)
    # 這個 dataset 物件現在包含原始欄位以及一個新的 'embedding' 欄位
    dataset_with_embeddings = transformer.convert()
except Exception as e:
    raise e

[2025-09-26 13:05:28,416: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-26 13:05:28,429: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-26 13:05:28,432: INFO: common: created directory at: artifacts]
[2025-09-26 13:05:28,434: INFO: common: created directory at: artifacts/data_embedding]
[2025-09-26 13:05:28,440: INFO: SentenceTransformer: Use pytorch device_name: cpu]
[2025-09-26 13:05:28,441: INFO: SentenceTransformer: Load pretrained SentenceTransformer: ibm-granite/granite-embedding-278m-multilingual]
[2025-09-26 13:05:36,326: INFO: 2931596709: Granite embedding model 'ibm-granite/granite-embedding-278m-multilingual' loaded successfully]
[2025-09-26 13:05:36,328: INFO: 2931596709: Loading dataset from 'artifacts/data_ingestion'...]
[2025-09-26 13:05:36,490: INFO: 2931596709: Generating embeddings for column 'dialogue'...]


Batches: 100%|██████████| 1/1 [00:14<00:00, 14.25s/it]

[2025-09-26 13:06:11,439: INFO: 2931596709: Encoded 32 texts into embeddings of shape (32, 768)]



Batches: 100%|██████████| 1/1 [00:11<00:00, 11.19s/it]xamples/s]

[2025-09-26 13:06:22,691: INFO: 2931596709: Encoded 32 texts into embeddings of shape (32, 768)]


...

[2025-09-26 14:34:03,845: INFO: 2931596709: Encoded 12 texts into embeddings of shape (12, 768)]



Map: 100%|██████████| 14732/14732 [1:28:06<00:00,  2.79 examples/s]

[2025-09-26 14:34:03,904: INFO: 2931596709: Saving dataset with embeddings to 'artifacts/data_embedding\samsum_dataset'...]



Saving the dataset (1/1 shards): 100%|██████████| 14732/14732 [00:00<00:00, 102019.89 examples/s]

[2025-09-26 14:34:04,083: INFO: 2931596709: Data embedding transformation complete.]
